# **Caso 3 — SQL: Libros, autores, editoriales, calificaciones y reseñas**

**Objetivos del estudio**
1. Conectar a la base de datos PostgreSQL provista.
2. Explorar las tablas (primeras filas) para validar estructura y tipos.
3. Responder, con SQL, las cinco tareas:
    - (Q1) Número de libros publicados después de 2000-01-01.
    - (Q2) Número de reseñas de usuarios y calificación promedio para cada libro.
    - (Q3) Editorial que ha publicado el mayor número de libros con más de 50 páginas.
    - (Q4) Autor con la calificación promedio más alta considerando solamente libros con al menos 50 calificaciones.
    - (Q5) Número promedio de reseñas de texto entre usuarios que han calificado más de 50 libros.

In [ ]:
# %% SQL_00_conexion

# Librerías
import pandas as pd
from sqlalchemy import create_engine

# Conectarse a la base de datos (usa CA.pem en la misma carpeta del .ipynb)
db_config = {
    'user': 'practicum_student',
    'pwd':  's65BlTKV3faNIGhmvJVzOqhs',
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432,
    'db':   'data-analyst-final-project-db'
}

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'], db_config['pwd'], db_config['host'], db_config['port'], db_config['db']
)

engine = create_engine(connection_string, connect_args={'sslmode': 'require'})


# **Explorar tablas**
Tablas: books, authors, publishers, ratings, reviews.
Se imprimen 5 filas de cada una para confirmar columnas y tipos.

In [ ]:
# %% SQL_01_head_tablas


books_head_dataframe      = pd.io.sql.read_sql("SELECT * FROM books LIMIT 5;", con=engine)
authors_head_dataframe    = pd.io.sql.read_sql("SELECT * FROM authors LIMIT 5;", con=engine)
publishers_head_dataframe = pd.io.sql.read_sql("SELECT * FROM publishers LIMIT 5;", con=engine)
ratings_head_dataframe    = pd.io.sql.read_sql("SELECT * FROM ratings LIMIT 5;", con=engine)
reviews_head_dataframe    = pd.io.sql.read_sql("SELECT * FROM reviews LIMIT 5;", con=engine)


# **(Q1) Número de libros publicados después del 1 de enero de 2000**
Filtro por publication_date::date > DATE '2000-01-01' y conteo.

In [ ]:
# %% SQL_02_Q1_libros_post_2000

query_q1 = """
SELECT COUNT(*) AS books_after_2000
FROM books
WHERE publication_date::date > DATE '2000-01-01';
"""
q1_result_dataframe = pd.io.sql.read_sql(query_q1, con=engine)
q1_result_dataframe

**Conclusión Q1**
books_after_2000 indica cuántos libros fueron publicados después de 2000-01-01.

# **(Q2) Número de reseñas de usuarios y calificación promedio para cada libro**
- Contar reseñas de texto por book_id desde reviews.
- Calcular promedio de rating por book_id desde ratings.
- Unir con books para mostrar title.

In [ ]:
# %% SQL_03_Q2_reviews_y_rating_por_libro

query_q2 = """
WITH reviews_count AS (
    SELECT
        book_id,
        COUNT(*) AS n_reviews
    FROM reviews
    GROUP BY book_id
),
ratings_avg AS (
    SELECT
        book_id,
        AVG(rating::float) AS avg_rating
    FROM ratings
    GROUP BY book_id
)
SELECT
    b.book_id,
    b.title,
    COALESCE(rc.n_reviews, 0)      AS n_reviews,
    ROUND(ra.avg_rating, 3)         AS avg_rating
FROM books b
LEFT JOIN reviews_count rc ON rc.book_id = b.book_id
LEFT JOIN ratings_avg  ra ON ra.book_id = b.book_id
ORDER BY b.book_id;
"""
q2_result_dataframe = pd.io.sql.read_sql(query_q2, con=engine)
q2_result_dataframe.head(10)


**Conclusión Q2**
Para cada libro se observa la cantidad de reseñas de texto y su calificación promedio (avg_rating).

# **(Q3) Editorial con más libros con más de 50 páginas**
- Filtrar books.num_pages > 50.
- Contar por editorial y devolver la editorial con el máximo.

In [ ]:
# %% SQL_04_Q3_editorial_con_mas_libros_50p

query_q3 = """
SELECT
    p.publisher_id,
    p.publisher,
    COUNT(*) AS n_books_over_50_pages
FROM books b
JOIN publishers p ON p.publisher_id = b.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher_id, p.publisher
ORDER BY n_books_over_50_pages DESC, p.publisher
LIMIT 1;
"""
q3_result_dataframe = pd.io.sql.read_sql(query_q3, con=engine)
q3_result_dataframe

**Conclusión Q3**
El resultado muestra la editorial con mayor número de libros > 50 páginas.

# **(Q4) Autor con mayor calificación promedio considerando solo libros con ≥ 50 calificaciones**
- Contar calificaciones por libro.
- Mantener libros con COUNT(rating_id) ≥ 50.
- Calcular el promedio de ratings del autor sobre ese subconjunto.

In [ ]:
# %% SQL_05_Q4_autor_mejor_promedio_con_threshold

query_q4 = """
WITH book_rating_stats AS (
    SELECT
        r.book_id,
        COUNT(r.rating_id) AS n_ratings
    FROM ratings r
    GROUP BY r.book_id
),
qualified_books AS (
    SELECT
        b.book_id,
        b.author_id
    FROM books b
    JOIN book_rating_stats s ON s.book_id = b.book_id
    WHERE s.n_ratings >= 50
),
author_avg AS (
    SELECT
        a.author_id,
        a.author,
        AVG(r.rating::float) AS author_avg_rating
    FROM qualified_books qb
    JOIN ratings r ON r.book_id = qb.book_id
    JOIN authors a ON a.author_id = qb.author_id
    GROUP BY a.author_id, a.author
)
SELECT
    author_id,
    author,
    ROUND(author_avg_rating, 3) AS author_avg_rating
FROM author_avg
ORDER BY author_avg_rating DESC, author
LIMIT 1;
"""
q4_result_dataframe = pd.io.sql.read_sql(query_q4, con=engine)
q4_result_dataframe

**Conclusión Q4**
Se muestra el autor con la mejor calificación promedio, usando solo libros con ≥ 50 calificaciones.

# **(Q5) Número promedio de reseñas de texto entre usuarios que calificaron más de 50 libros**
- Identificar usuarios con más de 50 libros calificados (distintos).
-  Contar reseñas de texto por usuario.
- Promediar (si un usuario no tiene reseñas de texto, cuenta como 0).

In [ ]:
# %% SQL_06_Q5_promedio_reviews_en_heavy_raters

query_q5 = """
WITH raters AS (
    SELECT
        username,
        COUNT(DISTINCT book_id) AS n_distinct_rated_books
    FROM ratings
    GROUP BY username
    HAVING COUNT(DISTINCT book_id) > 50
),
user_text_reviews AS (
    SELECT
        username,
        COUNT(*) AS n_text_reviews
    FROM reviews
    GROUP BY username
)
SELECT
    ROUND(AVG(COALESCE(utr.n_text_reviews, 0))::numeric, 3) AS avg_text_reviews_per_user
FROM raters r
LEFT JOIN user_text_reviews utr USING (username);
"""
q5_result_dataframe = pd.io.sql.read_sql(query_q5, con=engine)
q5_result_dataframe


**Conclusión Q5**
avg_text_reviews_per_user es el promedio de reseñas de texto entre usuarios que han calificado > 50 libros.

# **Fuentes y bibliografía (SQL)**

1. **PostgreSQL Documentation — SELECT / FROM / WHERE / GROUP BY / HAVING / ORDER BY**
https://www.postgresql.org/docs/current/queries.html
Consultas base usadas en todas las tareas (filtros, agrupaciones, ordenamientos).

2. **PostgreSQL — Aggregate Functions**
https://www.postgresql.org/docs/current/functions-aggregate.html
COUNT, AVG, ROUND en Q2–Q5.

3. **PostgreSQL — Joins**
https://www.postgresql.org/docs/current/queries-table-expressions.html#QUERIES-JOINS
JOIN y LEFT JOIN para combinar tablas (Q2–Q5).

4. **PostgreSQL — Common Table Expressions (WITH)**
https://www.postgresql.org/docs/current/queries-with.html
CTEs para estructurar pasos intermedios (Q2, Q4, Q5).

5. **Pandas — read_sql**
https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html
Ejecución de consultas SQL y visualización de resultados en el notebook.

6. **SQLAlchemy — Engine Basics**
https://docs.sqlalchemy.org/en/20/core/engines.html
Creación del engine y conexión a PostgreSQL con sslmode='require'.

7. **Material del Bootcamp (Sprints 6–9, 11–12)**
Joins, agregaciones, CTEs y reporte de resultados en notebooks.